### Seabed Mobility [2019, 2020, 2023, 2024]

#### Summary 
The SummaryThe code is used for seabed morphology and mobility analysis by comparing elevation data from different years (2019, 2020, 2023, 2024, 2025). It calculates differences between raster datasets, applies corrections, clips the results to buffer zones around wind turbine generator (WTG) locations, and computes zonal statistics. These statistics are then exported to Excel and CSV files for further analysis and reporting.

Main Steps in the Code:
#### 1) Environment Setup
Sets the workspace and enables output overwriting.Defines raster datasets (2019, 2020, 2023, 2024, 2025) as inputs. Sets the WTG layout file.
#### 2) Buffer Creation Around WTGs:
Creates a buffer zone of 200 meters around WTG locations using arcpy.Buffer_analysis.
#### 3) Raster Difference Calculation and Clipping:
Loops through pairs of raster datasets, calculates differences, applies corrections, and clips the resulting raster to the WTG buffer zone.
Zonal Statistics Calculation
#### 4) Calculates statistical metrics (Min, Max, Mean, Percentile 90) for each clipped raster within the buffer zone.
Saves the statistics into tables and exports them to Excel.
#### 4) Merges zonal statistics from multiple rasters into a single Excel file and then converts it into CSV.
#### 5) Joins the CSV data with the WTG shapefile based on common columns and creates a new shapefile with the results.

If the code should be change then:


## Posible  Future Changes: 

**1) New Raster Pair Definition:** 

*raster_2026= "Path to the raster"
raster_pairs = [...
            (raster_2026, raster_2019, "26_19.tif", 0.9),    
            ...]*
            
**2) Buffer distance:**

*buffer_distance = "100 meters"*     

**3) The code correctly applies mean corrections 0.9--> 1.5**

*raster_pairs = [...
            (raster_2026, raster_2019, "26_19.tif", 1.5), ...]*
            
**4) The fields MAX, MIN, MEAN, and PCT90 are hardcoded in the calculate_zonal_statistics function. e.g. 75th percentile directly**

*fields_to_keep = ["MAX", "MIN", "MEAN", "PCT75", abs_mean_field_name]*

In [ ]:
import arcpy
import numpy as np
import os
import pandas as pd

# Set environment
arcpy.env.overwriteOutput = True
workspace = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin"
arcpy.env.workspace = workspace

# Input rasters (update file paths once 2024 data is available)
raster_2019 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240206_from_WTP_004502963-03\004502963-03-MASP - GIS Data - Task 1 - Seabed Morphology and Mobility\RWE_Thor_Task1_v0.3.gdb\SN2019_011_R_DTM_MBES_100CM"
raster_2020 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240206_from_WTP_004502963-03\004502963-03-MASP - GIS Data - Task 1 - Seabed Morphology and Mobility\RWE_Thor_Task1_v0.3.gdb\MMT_628_GU_OWF_T50R_DTU15_MSL_DTM_100CM"
raster_2023 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\SITEINV\SITEINV.gdb\DK_THO_Fugro_MBES_2023_DEM_Mean_1pt00m_ra_UTM32N_EXT"
raster_2024 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240924_Bathy\F248460_RWE_Thor_1m_MSL\F248460_RWE_Thor_1m_MSL.tif"
raster_2025 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240924_Bathy\F248460_RWE_Thor_1m_MSL\F248460_RWE_Thor_1m_MSL.tif"


wtg = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_Layout_pt_UTM32N_v7"


# List of output difference raster names and correction values
raster_pairs = [
    (raster_2024, raster_2019, "24_19.tif", 0.9),     # 2024 - 2019
    (raster_2024, raster_2023, "24_23.tif", 0),     # 2024 - 2023
    (raster_2023, raster_2019, "23_19.tif", 0.11),  # 2023 - 2019 (+0.11)
    (raster_2023, raster_2020, "23_20.tif", 0.09),  # 2023 - 2020 (+0.09)
    (raster_2020, raster_2019, "20_19.tif", 0)      # 2020 - 2019 (no correction)
    (raster_2025, raster_2019, "20_19.tif", 0)
]

# Buffer parameters
buffer_distance = "200 meters"  # Example buffer size

# Functions

## A) Calculate difference rasters B) clip them to the buffer zone C) compute statistics

In [2]:
# Function to create a buffer around the WTG locations
def create_wtg_buffer(wtg_fc, buffer_distance, output_buffer):
    arcpy.Buffer_analysis(wtg_fc, output_buffer, buffer_distance)

# Combined function to calculate raster difference and clip to the buffer zone
def calculate_and_clip_difference_raster(raster1, raster2, buffer_fc, output_name, mean_correction=0):
    # Calculate raster difference
    diff_raster = arcpy.sa.Raster(raster1) - arcpy.sa.Raster(raster2)
    
    # Apply mean correction if provided
    if mean_correction != 0:
        diff_raster += mean_correction
    
    # Clip the difference raster to the buffer zone
    output_clipped_raster = os.path.join(workspace, output_name)
    arcpy.Clip_management(diff_raster, "#", output_clipped_raster, buffer_fc, "NoData", "ClippingGeometry")
    print(f"Raster clipped to buffer zone: {output_clipped_raster}")
    
    return output_clipped_raster

# Create buffer around WTG locations
wtg_buffer = os.path.join(workspace, "WTG_Buffer_200m.shp")
create_wtg_buffer(wtg, buffer_distance, wtg_buffer)

# Process rasters
for r1, r2, output_name, correction in raster_pairs:
    if arcpy.Exists(r1) and arcpy.Exists(r2):  # Only process if both rasters exist
        print(f"Processing difference and clipping for {output_name}...")
        calculate_and_clip_difference_raster(r1, r2, wtg_buffer, output_name, correction)
        


Processing difference and clipping for 24_19.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\24_19.tif
Processing difference and clipping for 24_23.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\24_23.tif
Processing difference and clipping for 23_19.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\23_19.tif
Processing difference and clipping for 23_20.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\23_20.tif
Processing difference and clipping for 20_19.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\20_19.tif


## Zonal statistics

In [4]:
# Function to calculate zonal statistics
def calculate_zonal_statistics(diff_raster, buffer_fc, zone_field, output_table, raster_name_prefix):

    # Shorten raster name prefix to a maximum of 6 characters for field name compliance
    raster_name_prefix = raster_name_prefix[:6]  # Ensures prefix is max 6 characters

    # Create Zonal Statistics tables for Max, Min, and Mean
    arcpy.sa.ZonalStatisticsAsTable(buffer_fc, zone_field, diff_raster, output_table, "DATA", "MIN", "MAX", "PCT90")
    
    # Add the field with the new shorter name for Abs Mean (keeping it under 10 characters)
    abs_mean_field_name = f"Abs_{raster_name_prefix}"
    arcpy.AddField_management(output_table, abs_mean_field_name, "DOUBLE")
    
    # Absolute value raster for mean of magnitude
    abs_diff_raster = arcpy.sa.Abs(diff_raster)
    
    # Calculate mean of magnitude (average of absolute values)
    abs_table = os.path.join(workspace, "abs" + os.path.basename(output_table))
    arcpy.sa.ZonalStatisticsAsTable(buffer_fc, zone_field, abs_diff_raster, abs_table, "DATA", "MEAN")
    
    # Ensure the correct field name is referenced for Abs Mean calculation
    with arcpy.da.UpdateCursor(output_table, [zone_field, abs_mean_field_name]) as cursor:
        with arcpy.da.SearchCursor(abs_table, [zone_field, "MEAN"]) as abs_cursor:
            abs_dict = {row[0]: row[1] for row in abs_cursor}  # Create a dictionary of zone ID and Abs Mean values
            for row in cursor:
                zone_id = row[0]
                if zone_id in abs_dict:
                    row[1] = abs_dict[zone_id]
                cursor.updateRow(row)
    
    # Read the output table into a DataFrame (only keep Max, Min, Mean, and Abs Mean fields)
    fields_to_keep = ["MAX", "MIN", "MEAN", "PCT90", abs_mean_field_name]
    df = pd.DataFrame.from_records(
        arcpy.da.TableToNumPyArray(output_table, [zone_field] + fields_to_keep)
    )  # Convert DBF to NumPy array and then to DataFrame
    
    # Rename columns with raster name prefix (keeping the fields under the 10 character limit)
    df.columns = [f"{col[:4]}_{raster_name_prefix}" if col != zone_field else col for col in df.columns] 
    
    # Change column names starting with 'Lay' to start with 'WTG'
    df.columns = [col.replace('Layout_ID1', 'WTG') if col.startswith('Lay') else col for col in df.columns]
    
    # Round all numeric columns to 3 decimal places
    numeric_cols = df.select_dtypes(include='number').columns  # Get numeric columns
    df[numeric_cols] = df[numeric_cols].round(3)  # Round to 3 decimal places
    
    # Export the DataFrame to an Excel file
    output_excel = os.path.join(workspace, os.path.basename(output_table).replace(".dbf", ".xlsx"))
    df.to_excel(output_excel, index=False, engine='openpyxl')

    print(f"Zonal statistics saved to {output_excel}")

# Zone field for WTG locations (use the field that represents unique WTG identifiers)
zone_field = "Layout_ID1"


# Initialize an empty DataFrame for merging
merged_df = pd.DataFrame()

# Iterate through raster pairs and calculate zonal statistics and join excel 
for r1, r2, output_name, correction in raster_pairs:
    if arcpy.Exists(r1) and arcpy.Exists(r2):  # Only process if both rasters exist
        print(f"Calculating zonal statistics for {output_name}...")
        output_table = os.path.join(workspace, os.path.basename(output_name).replace(".tif", ".dbf"))
        raster_name_prefix = os.path.splitext(os.path.basename(output_name))[0]  # Extract raster name for prefixing
        calculate_zonal_statistics(output_name, wtg_buffer, zone_field, output_table, raster_name_prefix)
        output_excel = os.path.join(workspace, os.path.basename(output_name).replace(".tif", ".xlsx"))
        df = pd.read_excel(output_excel)
         # Merge with the existing DataFrame on 'WTG' column
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='WTG', how='outer')
        
# Save the merged DataFrame to a new Excel file
merged_output_excel = os.path.join(workspace, "merged_zonal_statistics.xlsx")
merged_df.to_excel(merged_output_excel, index=False, engine='openpyxl')

print(f"Merged Excel file saved as: {merged_output_excel}")


Calculating zonal statistics for 24_19.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\24_19.xlsx
Calculating zonal statistics for 24_23.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\24_23.xlsx
Calculating zonal statistics for 23_19.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\23_19.xlsx
Calculating zonal statistics for 23_20.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\23_20.xlsx
Calculating zonal statistics for 20_19.tif...
Zonal statistics saved to \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\20_19.xlsx
Merged Excel file saved as: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_Justi

### Join Merged Stat excel file with WTG buffer shp

In [16]:
temp_csv_path = os.path.join(workspace, "merged_statistics.csv")

# Load the merged Excel into a DataFrame and convert to CSV
merged_df = pd.read_excel(merged_output_excel)

# Replace 0 in numeric columns with NaN
merged_df.loc[:, merged_df.select_dtypes(include='number').columns] = merged_df.select_dtypes(include='number').replace(0, np.nan)
merged_df.to_csv(temp_csv_path, index=False)

shapefile_join_column = "Layout_ID1"  # Column name in the shapefile
csv_join_column = "WTG"  # Column name in the CSV

# Add join
arcpy.management.AddJoin(wtg_buffer, shapefile_join_column, temp_csv_path, csv_join_column)

# Copy features to new shapefile
arcpy.management.CopyFeatures(wtg_buffer, "STAT")


<Result '\\\\WM20ocqu46ph01\\WF_Projects\\DK_THO\\4_OUTPUT\\SITEINV\\20241010_SeabedMobility_JustinPeterLarkin\\STAT.shp'>